In [7]:
import pandas as pd
import numpy as np
import os
import pycocotools.mask as mutils
import base64
import numpy as np
from pycocotools import mask as coco_mask_d
import typing as t
import zlib
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm


In [2]:
!ls ../

cas_275_epoch_1_0919pm.pth	preds_cas275_0919pm_all.pkl
configs				preds_cas_parent_0919pm_all.pkl
merge_0923_2_lb5026.csv		scripts
mmdetection			seg
notebooks			sub_htc275_lb4560_top100_iou03_mask42.csv
preds_0913am_all_lb4304.pkl	test.csv
preds_cas275_0917pm_lb4248.pkl	work_dirs
preds_cas275_0918pm_lb4279.pkl


In [3]:
df = pd.read_csv('../merge_0923_2_lb5026.csv')

In [10]:
df.head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,1024,683,/m/01k6s3 0.4164521 eNp9U8tuwjAQ/CU/4qD43CCF+N...
1,52a76f3fe21185bf,1024,683,/m/03y6mg 0.2951875 eNrNkm1vozAMx7+SHUgY7TadTq...
2,65e9da7c04c8e4cd,1024,682,/m/01lrl 0.0051426 eNqLyjUMNgxPMYACf0MDCoChH4...
3,6d10fa21d93bd6cc,1024,682,/m/01xyhv 0.9428729 eNp1UtFuwyAM/CUgkESb+rJKU0...
4,0841dee573a875ca,1024,683,/m/0342h 0.8243405 eNp1UUsOAjEIvRLQyUQvYGKmgrp...


In [8]:
def decode_mask(mask, h, w):
    mask = base64.b64decode(mask)
    mask = zlib.decompress(mask)
    decoding_dict = {
          'size': [h, w],
          'counts': mask
    }
    mask_tensor = coco_mask_d.decode(decoding_dict)

    #print(type(mask_tensor))
    return mask_tensor

In [39]:
def get_box_pred_str(row):
    pred_str = row.SegPredictionString
    h = row.ImageHeight
    w = row.ImageWidth
    try:
        if len(pred_str.strip()) < 1:
            return ''
    except:
        print('pred_str:', pred_str)
        raise
    det_items = pred_str.strip().split(' ')
    assert len(det_items) % 3 == 0

    res, det = [], []
    for i in range(len(det_items)):
        det.append(det_items[i])
        if (i+1) % 3 == 0:
            assert '/' in det[0]
            mask = decode_mask(det[2], h, w)
            b = cv2.boundingRect(mask)
            bbox = [b[0]/w, b[1]/h, (b[0]+b[2])/w, (b[1]+b[3])/h]
            bbox = ['{:.5f}'.format(x) for x in bbox]
            res.append(det[0])
            res.append(det[1])
            res.extend(bbox)
            det = []
    return ' '.join(res)

In [11]:
df.rename(columns={'PredictionString': 'SegPredictionString'}, inplace=True)

In [29]:
df.head()

,ImageID,ImageWidth,ImageHeight,SegPredictionString
0,80155d58d0ee19bd,1024,683,/m/01k6s3 0.4164521 eNp9U8tuwjAQ/CU/4qD43CCF+N...
1,52a76f3fe21185bf,1024,683,/m/03y6mg 0.2951875 eNrNkm1vozAMx7+SHUgY7TadTq...
2,65e9da7c04c8e4cd,1024,682,/m/01lrl 0.0051426 eNqLyjUMNgxPMYACf0MDCoChH4...
3,6d10fa21d93bd6cc,1024,682,/m/01xyhv 0.9428729 eNp1UtFuwyAM/CUgkESb+rJKU0...
4,0841dee573a875ca,1024,683,/m/0342h 0.8243405 eNp1UUsOAjEIvRLQyUQvYGKmgrp...


In [40]:
def set_pred_str(df):
    df['PredictionString'] = df.apply(lambda row: get_box_pred_str(row), axis=1)
    return df

In [41]:
from multiprocessing import Pool

def parallel_apply(df, func, n_cores=24):
    #ncores = 24
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [42]:
df = parallel_apply(df, set_pred_str)

In [35]:
df.head()

,ImageID,ImageWidth,ImageHeight,SegPredictionString,PredictionString
0,80155d58d0ee19bd,1024,683,/m/01k6s3 0.4164521 eNp9U8tuwjAQ/CU/4qD43CCF+N...,/m/01k6s3 0.4164521 269312.00000 125672.00000 ...
1,52a76f3fe21185bf,1024,683,/m/03y6mg 0.2951875 eNrNkm1vozAMx7+SHUgY7TadTq...,/m/03y6mg 0.2951875 649216.00000 21856.00000 1...
2,65e9da7c04c8e4cd,1024,682,/m/01lrl 0.0051426 eNqLyjUMNgxPMYACf0MDCoChH4...,/m/01lrl 0.0051426 2048.00000 407154.00000 150...
3,6d10fa21d93bd6cc,1024,682,/m/01xyhv 0.9428729 eNp1UtFuwyAM/CUgkESb+rJKU0...,/m/01xyhv 0.9428729 557056.00000 116622.00000 ...
4,0841dee573a875ca,1024,683,/m/0342h 0.8243405 eNp1UUsOAjEIvRLQyUQvYGKmgrp...,/m/0342h 0.8243405 481280.00000 232220.00000 7...


In [43]:
#df.PredictionString = ''
df.head()
 

,ImageID,ImageWidth,ImageHeight,SegPredictionString,PredictionString
0,80155d58d0ee19bd,1024,683,/m/01k6s3 0.4164521 eNp9U8tuwjAQ/CU/4qD43CCF+N...,/m/01k6s3 0.4164521 0.25684 0.26940 0.72363 0....
1,52a76f3fe21185bf,1024,683,/m/03y6mg 0.2951875 eNrNkm1vozAMx7+SHUgY7TadTq...,/m/03y6mg 0.2951875 0.61914 0.04685 0.97559 0....
2,65e9da7c04c8e4cd,1024,682,/m/01lrl 0.0051426 eNqLyjUMNgxPMYACf0MDCoChH4...,/m/01lrl 0.0051426 0.00195 0.87537 0.14355 0.9...
3,6d10fa21d93bd6cc,1024,682,/m/01xyhv 0.9428729 eNp1UtFuwyAM/CUgkESb+rJKU0...,/m/01xyhv 0.9428729 0.53125 0.25073 0.88379 0....
4,0841dee573a875ca,1024,683,/m/0342h 0.8243405 eNp1UUsOAjEIvRLQyUQvYGKmgrp...,/m/0342h 0.8243405 0.45898 0.49780 0.71582 0.7...


In [47]:
df.to_csv('seg_lb5026_bbox.csv', index=False, columns=['ImageID', 'PredictionString'])

In [44]:
df.iloc[0].PredictionString

'/m/01k6s3 0.4164521 0.25684 0.26940 0.72363 0.80673 /m/025dyy 0.1017320 0.25977 0.26940 0.72266 0.80234 /m/01s55n 0.0560076 0.25977 0.26940 0.72266 0.80673 /m/0lt4_ 0.0544400 0.25586 0.26794 0.72363 0.80527 /m/0dv5r 0.0509178 0.26074 0.26794 0.72266 0.80673 /m/03bbps 0.0356407 0.25781 0.26794 0.72363 0.80673 /m/02tsc9 0.0329684 0.25586 0.26647 0.72363 0.80381 /m/0bjyj5 0.0314410 0.25684 0.26940 0.72363 0.80673 /m/029b3 0.0260458 0.25684 0.26794 0.72363 0.79356 /m/06_72j 0.0258255 0.26074 0.26940 0.72363 0.79795 /m/046dlr 0.0248863 0.25684 0.26940 0.72266 0.79795 /m/025dyy 0.0222434 0.38574 0.65593 0.69922 0.82138 /m/0gjkl 0.0221946 0.25781 0.26647 0.72266 0.80234 /m/03y6mg 0.0194654 0.25684 0.26794 0.72266 0.80673 /m/04h8sr 0.0176861 0.25586 0.26794 0.72461 0.79356 /m/01kb5b 0.0133547 0.25586 0.26794 0.72461 0.80088 /m/020lf 0.0121866 0.25781 0.26794 0.72363 0.80381 /m/01226z 0.0092463 0.26270 0.26794 0.71973 0.73792 /m/03s_tn 0.0086645 0.25684 0.26647 0.72363 0.80527 /m/03m3vtv 0.008